In [ ]:
%pip install datasets optuna -q

# Task 1

In [ ]:
from datasets import Dataset
from datasets.formatting.formatting import LazyBatch
import nest_asyncio
import numpy as np
from numpy.typing import NDArray
import optuna
from optuna.visualization import plot_param_importances, plot_contour
import pandas as pd
from plotly.graph_objects import Figure
from sklearn.preprocessing import LabelEncoder
from transformers.models.auto.modeling_auto import AutoModelForSequenceClassification
from transformers.models.auto.tokenization_auto import AutoTokenizer
from transformers.modeling_utils import PreTrainedModel
from transformers.tokenization_utils_base import BatchEncoding, PreTrainedTokenizerBase
from transformers.trainer import Trainer
from transformers.trainer_callback import EarlyStoppingCallback
from transformers.trainer_utils import get_last_checkpoint
from transformers.training_args import TrainingArguments
import wandb

from modules import dataset, models_creation, paths
from modules.utils import model as model_utils

In [ ]:
# Allows for asyncio to be run in notebooks
nest_asyncio.apply()

In [ ]:
model_name: str = 'google/electra-small-discriminator'

## Data

### Basic Operations

In [ ]:
# Load the dataset
full_train_set: pd.DataFrame = dataset.extract_dataset('train')
full_val_set: pd.DataFrame = dataset.extract_dataset('validation')
full_test_set: pd.DataFrame = dataset.extract_dataset('test')

In [ ]:
# Extract x and y
train_x: pd.DataFrame = full_train_set[['item', 'description', 'enwiki_extract']]
train_y: pd.Series = full_train_set['label']
val_x: pd.DataFrame = full_val_set[['item', 'description', 'enwiki_extract']]
val_y: pd.Series = full_val_set['label']
test_x: pd.DataFrame = full_test_set[['item', 'description', 'enwiki_extract']]

### Preprocessing

In [ ]:
# Encode labels
label_encoder: LabelEncoder = LabelEncoder()
train_y_encoded: pd.Series = pd.Series(label_encoder.fit_transform(train_y),    # type: ignore
                                       name = train_y.name,
                                       index = train_y.index
                                       )
val_y_encoded: pd.Series = pd.Series(label_encoder.transform(val_y),    # type: ignore
                                     name = val_y.name,
                                     index = val_y.index
                                     )

In [ ]:
# Merge the dataframes
train_df: pd.DataFrame = pd.concat([train_x, train_y_encoded], axis = 1)
val_df: pd.DataFrame = pd.concat([val_x, val_y_encoded], axis = 1)
test_df: pd.DataFrame = test_x

In [ ]:
# Define the tokenizer
tokenizer: PreTrainedTokenizerBase = AutoTokenizer.from_pretrained(model_name)

# Add the special tokens
tokenizer.add_special_tokens({'additional_special_tokens': ['<item>', '<description>', '<enwiki>']})    # type: ignore

In [ ]:
# Tokenize the data
def tokenize(examples: LazyBatch) -> BatchEncoding:
    """
    Tokenizes the input data by concatenating the item, description, and enwiki_extract fields and separating them with the special tokens.
    """
    
    results: list[str] = []
    for item, description, enwiki in zip(examples['item'], examples['description'], examples['enwiki_extract']):    # type: ignore
        # Concatenate the item, description, and enwiki_extract fields
        results.append(f"<item> {item} <description> {description} <enwiki> {enwiki}")

    return tokenizer(results, truncation = True)

train_data: Dataset = Dataset.from_pandas(train_df).map(tokenize, batched = True)
val_data: Dataset = Dataset.from_pandas(val_df).map(tokenize, batched = True)
test_data: Dataset = Dataset.from_pandas(test_df).map(tokenize, batched = True)

## Model

### Tuning and Training

In [ ]:
# Load the pretrained model
pretrained_model: PreTrainedModel = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                                                       num_labels = len(label_encoder.classes_),
                                                                                       ignore_mismatched_sizes = True
                                                                                       )

In [ ]:
# Resize the model embeddings
pretrained_model.resize_token_embeddings(len(tokenizer))

In [ ]:
def objective(trial: optuna.Trial) -> float:
    """
    Objective function for Optuna to optimize the hyperparameters.
    """

    # Hyperparameters
    learning_rate: float = trial.suggest_float('learning_rate', 1e-6, 1e-4, log = True)
    weight_decay: float = trial.suggest_float('weight_decay', 1e-6, 0.1, log = True)
    warmup_steps: int = trial.suggest_int('warmup_steps', 0, 1000, step = 100)

    # Initialize wandb
    model_utils.configure_wandb(project = 'Cultural classification on text',
                                name = f'trial_{trial.number}',
                                library = 'transformers'
                                )

    # Trainer
    trainargs: TrainingArguments = TrainingArguments(num_train_epochs = 1000,    # Unlimited epochs
                                                     learning_rate = learning_rate,
                                                     weight_decay = weight_decay,
                                                     warmup_steps = warmup_steps,
                                                     auto_find_batch_size = True,
                                                     fp16 = True,
                                                     metric_for_best_model = 'f1',
                                                     load_best_model_at_end = True,
                                                     eval_strategy = 'steps',
                                                     eval_steps = 100,
                                                     save_strategy = 'best',
                                                     save_total_limit = 1,
                                                     output_dir = str(paths.TRANSFORMER_MODEL_DIR),
                                                     report_to = 'wandb'
                                                     )

    trainer: Trainer = Trainer(callbacks = [EarlyStoppingCallback(early_stopping_patience = 10)],
                               model = pretrained_model,
                               args = trainargs,
                               train_dataset = train_data,
                               eval_dataset = val_data,
                               processing_class = tokenizer,
                               compute_metrics = models_creation.transformer_metrics
                               )

    # Train the model   
    trainer.train()

    # Evaluate the model
    f1: float
    try:
        f1 = wandb.summary['eval/f1']['max']    # type: ignore
    except:
        raise optuna.TrialPruned("No f1 score found in wandb logger. Probably something went wrong during training.")
    finally:
        # Close the wandb run
        wandb.finish()

    return f1

In [ ]:
# Optuna study
study: optuna.Study = optuna.create_study(direction = 'maximize')
study.optimize(objective, n_trials = 10, show_progress_bar = True)

In [ ]:
# Plot parameter importances
param_importances_fig: Figure = plot_param_importances(study)
param_fig: Figure = plot_param_importances(study)
param_fig.update_layout(autosize = False,
                        width = 1200,
                        height = 400
                        )
param_fig.show()

# Plot contour
contour_fig: Figure = plot_contour(study)
contour_fig.update_layout(autosize = False,
                          width = 1200,
                          height = 1200
                          )
contour_fig.show()

In [ ]:
# Print the best hyperparameters
best_params: dict[str, int|float] = study.best_trial.params
best_lr: float = best_params['learning_rate']
best_weight_decay: float = best_params['weight_decay']
best_warmup_steps: float = best_params['warmup_steps']

print(f"""Best hyperparameters:
      \tlearning rate: {best_lr:.3e}
      \tweight decay: {best_weight_decay:.3e}
      \twarmup steps: {best_warmup_steps}"""
      )

### Results

In [ ]:
# Load the model
model: PreTrainedModel = AutoModelForSequenceClassification.from_pretrained(get_last_checkpoint(paths.TRANSFORMER_MODEL_DIR))

In [ ]:
# Initialize the trainer
results_trainer: Trainer = Trainer(model = model,
                                   args = TrainingArguments(auto_find_batch_size = True,
                                                            output_dir = str(paths.TRANSFORMER_MODEL_DIR),
                                                            report_to = 'none'
                                                            ),
                                   eval_dataset = val_data,
                                   processing_class = tokenizer,
                                   compute_metrics = models_creation.transformer_metrics
                                   )

#### Validation

In [ ]:
# Evaluate the model on the validation set
val_results: dict[str, float] = results_trainer.evaluate()
print(f"Loss: {val_results['eval_loss']:.3f}")
print(f"Accuracy: {val_results['eval_accuracy']:.3f}")
print(f"F1 score: {val_results['eval_f1']:.3f}")
print(f"Precision: {val_results['eval_precision']:.3f}")
print(f"Recall: {val_results['eval_recall']:.3f}")

In [ ]:
# Confusion matrix
val_logits: NDArray[np.float32] = np.array(results_trainer.predict(val_data).predictions)    # type: ignore
val_predictions_encoded: NDArray[np.intp] = np.argmax(val_logits, axis = 1)
model_utils.plot_confusion_matrix(val_y_encoded, val_predictions_encoded, label_encoder)

#### Test

In [ ]:
# Get the predictions on the test set
test_logits: NDArray[np.float32] = np.array(results_trainer.predict(test_data).predictions)    # type: ignore
test_predictions_encoded: NDArray[np.intp] = np.argmax(test_logits, axis = 1)
test_predictions: NDArray[str] = label_encoder.inverse_transform(test_predictions_encoded)  # type: ignore

# Save the predictions
test_predictions_df: pd.DataFrame = pd.DataFrame({'item': full_test_set['item'],
                                                  'name': full_test_set['name'],
                                                  'label': test_predictions
                                                  }, index = full_test_set.index
                                                  )
test_predictions_df.to_csv(paths.TRANSFORMER_PREDICITONS, index_label = 'id')
print(f"Saved the predicitons on test set to {paths.TRANSFORMER_PREDICITONS}")